In [1]:
import sys
import random 
import requests
from time import sleep

import numpy as np

# assumes working directory is notebook location
corepath = "../../../helao-core"
sys.path.append(corepath)
from helao.core.models.experiment import ExperimentTemplate

sys.path.append("../..")
from helao.helpers.premodels import Sequence
from helao.configs.simulate import config as global_cfg

cfg = global_cfg['servers']

In [2]:
palhost = cfg["PAL"]["host"]
palport = cfg["PAL"]["port"]
orchhost = cfg["ORCH"]["host"]
orchport = cfg["ORCH"]["port"]

# request explorable space

In [3]:
# use robotic sampler 'PAL' action server as placeholder for sample database server
resp = requests.post(f"http://{palhost}:{palport}/list_all_spaces")
resp.status_code == 200

True

In [4]:
# available composition,pH spaces
sorted([(d['elements'], d['solution_ph']) for d in resp.json()])

[(['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 3),
 (['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 7),
 (['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 9),
 (['Co', 'Mn', 'Sn', 'Mg', 'Zn', 'Ca'], 13),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 3),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 7),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 9),
 (['Fe', 'Co', 'Ta', 'Mn', 'Cu', 'Sn'], 13),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 3),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 7),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 9),
 (['Ni', 'Fe', 'Co', 'Ta', 'Mn', 'Cu'], 13),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 3),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 7),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 9),
 (['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'], 13)]

### terminology
- a 'sequence' is a queue of experiments performed in order
- an 'experiment' is a queue of actions performed in order
- _side note:_ one or more ESAMP processes may be created from an experiment, the experiment `SIM_measure_CP` produces 1 process
- an 'action' is the atomic request dispatched by the Orchestrator to individual action servers
- an 'action server' exposes hardware driver and data management functions via FastAPI
- an 'Orchestrator' manages the queing and dispatch sequences, experiments, and action requests

In [5]:
# valid experiment names
import helao.experiments.simulate_exp
helao.experiments.simulate_exp.__all__

['SIM_measure_CP']

In [6]:
from helao.experiments.simulate_exp import SIM_measure_CP
SIM_measure_CP

<function helao.experiments.simulate_exp.SIM_measure_CP(experiment: helao.helpers.premodels.Experiment, experiment_version: int = 1, solution_ph: Optional[int] = 13, elements: Optional[List[str]] = [], element_fracs: Optional[List[float]] = [])>

- `SIM_measure_CP` is the only experiment available in this simulator
- `SIM_measure_CP` has 3 real arguments: `solution_ph`, `elements`, and `element_fracs`
- args `experiment` and `experiment_version` are managed by orchestrator

### `SIM_measure_CP` experiment performs 9 actions:
1. query available plates for elements (and pH) matching `solution_ph` and `elements`
2. load plate_id identified in (1)
3. query available samples for element fractions matching `element_fracs`
4. locate x,y coordinates for sample identified in (3)
5. move stage motors to x,y, coordinates identified in (4)
6. run CP measurement at 3 mA/cm2 for 15 seconds
7. extract Eta (V vs O2/H2O) from measurement in (6)
8. run CP measurement at 10 mA/cm2 for 15 seconds
9. extract Eta (V vs O2/H2O) from measurement in (9)

# example Ni-Fe-La-Ce-Co-Ta @ pH=13

In [7]:
# get addressable composition space (X's) from previous request
elements = ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta']
solution_ph = 13

comp_space = [x for x in resp.json() if x['elements']==elements and x['solution_ph']==solution_ph][0]['element_fracs']
len(comp_space)

2051

In [8]:
comp_space[:5]

[[0.1, 0.2, 0.0, 0.0, 0.6, 0.1],
 [0.0, 0.3, 0.0, 0.2, 0.5, 0.0],
 [0.2, 0.0, 0.1, 0.0, 0.4, 0.3],
 [0.1, 0.0, 0.4, 0.1, 0.4, 0.0],
 [0.2, 0.1, 0.2, 0.0, 0.0, 0.5]]

In [9]:
# initial random seed of 5 compositions
random.seed(0)
comp_inds = random.sample(range(len(comp_space)), 5)

# create sequence object for holding experiments
sequence = Sequence(sequence_name='seed_sequence')

# populate sequence's experiment list
for i in comp_inds:
    sequence.experiment_plan_list.append(
        ExperimentTemplate(
            experiment_name="SIM_measure_CP",
            experiment_params={
                "solution_ph": 13,
                "elements": ["Ni", "Fe", "La", "Ce", "Co", "Ta"],
                "element_fracs": comp_space[i],
            },
        )
    )

In [10]:
# preview sequence object, identifying info such as uuid and timestamp are only created when a sequence is dispactched (executed by Orchestrator)
sequence.as_dict()

{'sequence_name': 'seed_sequence',
 'sequence_params': {},
 'sequence_label': 'noLabel',
 'experiment_plan_list': [{'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.1, 0.8, 0.0, 0.0, 0.1]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.3, 0.3, 0.0, 0.4, 0.0]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.0, 0.2, 0.0, 0.2, 0.6]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
    'element_fracs': [0.0, 0.0, 0.4, 0.3, 0.2, 0.1]}},
  {'experiment_name': 'SIM_measure_CP',
   'experiment_params': {'solution_ph': 13,
    'elements': ['Ni', 'Fe', 'L

In [11]:
# send sequence to Orchestrator
seq_req = requests.post(f"http://{orchhost}:{orchport}/append_sequence", json={"sequence": sequence.as_dict()})
seq_req.status_code == 200  # successful post request

True

In [12]:
# get list of loaded sequences on Orchestrator
orch_list = requests.post(f"http://{orchhost}:{orchport}/list_sequences")
orch_list.status_code == 200  # successful post request

True

In [13]:
orch_list.json() # present sequence queue

[{'sequence_name': 'seed_sequence',
  'sequence_params': {},
  'sequence_label': 'noLabel',
  'experiment_plan_list': [{'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.1, 0.8, 0.0, 0.0, 0.1]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.3, 0.3, 0.0, 0.4, 0.0]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.0, 0.2, 0.0, 0.2, 0.6]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elements': ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'],
     'element_fracs': [0.0, 0.0, 0.4, 0.3, 0.2, 0.1]}},
   {'experiment_name': 'SIM_measure_CP',
    'experiment_params': {'solution_ph': 13,
     'elem

In [14]:
# start Orch (begin or resume dispatching sequence/experiment/action queues)
orch_start = requests.post(f"http://{orchhost}:{orchport}/start")
orch_start.status_code == 200

True

__Notes on Orch status:__

The orchestrator server holds minimal state variables, so we can only ask:
1. whether it's currently stopped or running
2. the state of dispatched actions of the active experiment

When Orch completes all queued actions, experiments, and sequences, the states in (2) will be cleared.

_Ideally an experiment would use a final action that pushes a message to GCLD._ In lieu of this, we can set up a primitive polling loop to track running state and count the number of dispatched experiments.

In [15]:
orch_status = requests.post(f"http://{orchhost}:{orchport}/get_status")

dispatched_exps = set()
last_exp_count = 0
while orch_status.json()["loop_state"] == "started":
    sleep(2)
    orch_status = requests.post(f"http://{orchhost}:{orchport}/get_status")
    active_dict = orch_status.json()['active_dict']
    for act_uuid, act_dict in active_dict.items():
        dispatched_exps.add(act_dict['act']['experiment_uuid'])
    if len(dispatched_exps) != last_exp_count:
        last_exp_count = len(dispatched_exps)
        print(last_exp_count, "experiments have been dispatched.")
print("Orch has stopped.")

1 experiments have been dispatched.
2 experiments have been dispatched.
3 experiments have been dispatched.
4 experiments have been dispatched.
5 experiments have been dispatched.
Orch has stopped.


In [16]:
# use robotic sampler 'PAL' action server as placeholder for sample database server
acq_resp = requests.post(f"http://{palhost}:{palport}/get_measured", json={"start_idx": 0})
acq_resp.status_code == 200

True

In [17]:
acq_resp.json()

[{'Ni': 0.0,
  'Fe': 0.1,
  'La': 0.8,
  'Ce': 0.0,
  'Co': 0.0,
  'Ta': 0.1,
  'solution_ph': 13,
  'eta3': 0.411175,
  'eta10': 0.502737},
 {'Ni': 0.0,
  'Fe': 0.3,
  'La': 0.3,
  'Ce': 0.0,
  'Co': 0.4,
  'Ta': 0.0,
  'solution_ph': 13,
  'eta3': 0.361331,
  'eta10': 0.397913},
 {'Ni': 0.0,
  'Fe': 0.0,
  'La': 0.2,
  'Ce': 0.0,
  'Co': 0.2,
  'Ta': 0.6,
  'solution_ph': 13,
  'eta3': 0.41637,
  'eta10': 0.462762},
 {'Ni': 0.0,
  'Fe': 0.0,
  'La': 0.4,
  'Ce': 0.3,
  'Co': 0.2,
  'Ta': 0.1,
  'solution_ph': 13,
  'eta3': 0.395245,
  'eta10': 0.456316},
 {'Ni': 0.2,
  'Fe': 0.0,
  'La': 0.3,
  'Ce': 0.5,
  'Co': 0.0,
  'Ta': 0.0,
  'solution_ph': 13,
  'eta3': 0.417371,
  'eta10': 0.494323}]

In [18]:
# acquire new batch of 5 random comps
random.seed(0)
batch2_inds = random.sample(range(len(comp_space)), 10)[5:]
batch2_inds

[1658, 1242, 1952, 1466, 894]

In [19]:
# original seed indices
comp_inds

[1577, 1722, 165, 1060, 1990]

In [20]:
# create sequence object for holding experiments
sequence = Sequence(sequence_name='seed_sequence')

# populate sequence's experiment list
for i in batch2_inds:
    sequence.experiment_plan_list.append(
        ExperimentTemplate(
            experiment_name="SIM_measure_CP",
            experiment_params={
                "solution_ph": 13,
                "elements": ["Ni", "Fe", "La", "Ce", "Co", "Ta"],
                "element_fracs": comp_space[i],
            },
        )
    )

In [21]:
# send sequence to Orchestrator
seq_req = requests.post(f"http://{orchhost}:{orchport}/append_sequence", json={"sequence": sequence.as_dict()})
seq_req.status_code == 200  # successful post request

True

In [22]:
# start Orch (begin or resume dispatching sequence/experiment/action queues)
orch_start = requests.post(f"http://{orchhost}:{orchport}/start")
orch_start.status_code == 200

True

In [23]:
orch_status = requests.post(f"http://{orchhost}:{orchport}/get_status")

dispatched_exps = set()
last_exp_count = 0
while orch_status.json()["loop_state"] == "started":
    sleep(2)
    orch_status = requests.post(f"http://{orchhost}:{orchport}/get_status")
    active_dict = orch_status.json()['active_dict']
    for act_uuid, act_dict in active_dict.items():
        dispatched_exps.add(act_dict['act']['experiment_uuid'])
    if len(dispatched_exps) != last_exp_count:
        last_exp_count = len(dispatched_exps)
        print(last_exp_count, "experiments have been dispatched.")
print("Orch has stopped.")

1 experiments have been dispatched.
2 experiments have been dispatched.
3 experiments have been dispatched.
4 experiments have been dispatched.
5 experiments have been dispatched.
Orch has stopped.


In [24]:
# change 'start_idx' query parameter to slice list of measured space
start_idx = 5
acq2_resp = requests.post(f"http://{palhost}:{palport}/get_measured?start_idx={start_idx}")
acq2_resp.status_code == 200

True

In [25]:
acq2_resp.json()

[{'Ni': 0.1,
  'Fe': 0.0,
  'La': 0.0,
  'Ce': 0.1,
  'Co': 0.8,
  'Ta': 0.0,
  'solution_ph': 13,
  'eta3': 0.379036,
  'eta10': 0.420298},
 {'Ni': 0.2,
  'Fe': 0.0,
  'La': 0.2,
  'Ce': 0.0,
  'Co': 0.3,
  'Ta': 0.3,
  'solution_ph': 13,
  'eta3': 0.395399,
  'eta10': 0.440602},
 {'Ni': 0.2,
  'Fe': 0.0,
  'La': 0.3,
  'Ce': 0.0,
  'Co': 0.0,
  'Ta': 0.5,
  'solution_ph': 13,
  'eta3': 0.373508,
  'eta10': 0.42117},
 {'Ni': 0.1,
  'Fe': 0.0,
  'La': 0.2,
  'Ce': 0.1,
  'Co': 0.0,
  'Ta': 0.6,
  'solution_ph': 13,
  'eta3': 0.403961,
  'eta10': 0.458812},
 {'Ni': 0.0,
  'Fe': 0.2,
  'La': 0.5,
  'Ce': 0.1,
  'Co': 0.2,
  'Ta': 0.0,
  'solution_ph': 13,
  'eta3': 0.372126,
  'eta10': 0.407029}]

In [26]:
# further sequences/experiments on the same space/plate will aggregate results on the PAL server
# loading a new space (i.e. issuing a new experiment with different elements+pH from previous) will reset the aggregated results
# results should be queried prior to changing plates

# the following request manually resets the list of acquired samples on the PAL server
reset_resp = requests.post(f"http://{palhost}:{palport}/clear_measured")
reset_resp.status_code == 200

True